In [1]:
import os

In [2]:
%pwd

'e:\\Projects\\ETE Project\\WineIQ-A-Machine-Learning-Approach-to-Quality-Scoring\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\Projects\\ETE Project\\WineIQ-A-Machine-Learning-Approach-to-Quality-Scoring'

In [5]:
# create entity
# entity is a collection of attributes that define a specific object or concept
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [ ]:
from MLProject.constants import *

In [7]:
from MLProject.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [10]:
import os
import urllib.request as request
import zipfile
from MLProject import logger
from MLProject.utils.common import get_size

In [11]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} Download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")


    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [12]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-06-25 21:29:06,290: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-06-25 21:29:06,705: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-25 21:29:06,748: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-06-25 21:29:06,751: INFO: common: created directory at: artifacts]
[2025-06-25 21:29:06,754: INFO: common: created directory at: artifacts/data_ingestion]
[2025-06-25 21:29:21,001: INFO: 2948980364: artifacts/data_ingestion/data.zip Download! with following info: 
Connection: close
Content-Length: 24777
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "4ef28118b018e953379f4f6ac8522a396c91053044158228bc10d5989399519b"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: D0F3:FE811:8E7E:12C01:685C1CD8
Accept-Ranges: bytes
Date: Wed, 25 Jun